## import

In [88]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
# import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression

## some sheets

In [ ]:
# text1 = 'mom washed the frame'
# text2 = 'mom shook her tits'
# text3 = 'mom broke the window and broke the frame by her tits'

# vocab = ['mom', 'washed', 'the', 'frame', 'shook', 'her', 'tits', 'broke', 'window', 'and', 'by']
# vector_text1 = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
# vector_text2 = [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
# vector_text3 = [1, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1]


In [ ]:
# corpus = [
#     text1,
#     text2,
#     text3
# ]
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
# vectorizer.get_feature_names()


# print(X.toarray())

## Day 1

In [ ]:
# BOT_CONFIG = {
#     'intents':{
#         'hello':{
#             'examples':['hi', 'hello', 'howdoyoudo'],
#             'responses':['good day', 'Hello', 'watsup'],
#           },
#         'bye':{
#             'examples':['bye', 'gOOd bye', 'fuck off'],
#             'responses':['good bye', 'see you', 'go fuck yourself'],
#           }  
#         }, 
#     'not_found' : {
#       'responses':'i don\'t understand you',
#     }
# }


In [ ]:
# def clean(text):
#   cleaned_text = ''
#   for ch in text.lower():
#     if ch in 'abcdefghijklmnopqrstuvwxyz ':
#       cleaned_text = cleaned_text + ch  
#   return cleaned_text


# def get_intent(text):
#   for intent in BOT_CONFIG['intents'].keys():
#     for example in BOT_CONFIG['intents'][intent]['examples']:
#       cleaned_example = clean(example)
#       cleaned_text = clean(text) 
#       if nltk.edit_distance(cleaned_example, cleaned_text) / max(len(cleaned_text), len(cleaned_example)) < 0.4:
#         return intent
#   return 'not_found'


# def bot(text):
#   intent = get_intent(text)
#   if intent != 'not_found':
#     return random.choice(BOT_CONFIG['intents'][intent]['responses'])
#   else:
#     return BOT_CONFIG['not_found']['responses']

In [ ]:
# input_text = ''
# while input_text != 'exit':
#   input_text = input()
#   response = bot(input_text)
#   print(response)

## Day 2

In [89]:
with open ('/content/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)

# Новый раздел

In [ ]:
f

In [90]:
X = []
y = []
for intent in BOT_CONFIG['intents']:
  for example in BOT_CONFIG['intents'][intent]['examples']:
    X.append(example)
    y.append(intent)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y

In [ ]:
X

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char_wb')
# vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

len(vectorizer.get_feature_names())


In [ ]:
clf = RandomForestClassifier()
# clf = LogisticRegression(C=5, max_iter=300)
clf.fit(X_train_vectorized, y_train)

In [ ]:
clf.score(X_train_vectorized, y_train)

In [ ]:
clf.score(X_test_vectorized, y_test)

In [ ]:
# clf.predict(vectorizer.transform(['hi']))

## testing

In [95]:
def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

def bot(text):
    intent = get_intent_by_model(text)
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
input_text = ''
while input_text != clean('exit'):
   input_text = input()
   response = bot(input_text)
   print(response)

## deploy to telegramm

In [ ]:
# !pip install python-telegram-bot --upgrade      # evуry time reinstall

In [97]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    uot_text = bot(update.message.text)
    update.message.reply_text(uot_text)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("2066634680:AAGBET2prIZZ19Pm0NXF-dsU77F-Eu7IzPY")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [98]:
main()

2021-10-30 17:08:53,322 - apscheduler.scheduler - INFO - Scheduler started
2021-10-30 17:22:05,060 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-10-30 17:22:05,063 - apscheduler.scheduler - INFO - Scheduler has been shut down
